# Naive Bayes 분류기 구현하기

이 노트북에서는 Naive Bayes 분류기를 처음부터 구현해보겠습니다.
햄릿 텍스트를 사용하여 문장에 'be'라는 단어가 포함되는지 여부를 분류하는 이진 분류 모델을 만들어보겠습니다.


## 1. 필요한 라이브러리 및 유틸리티 함수 import


In [1]:
from utils import process_utt
from utils import lookup
from nltk.corpus.reader import PlaintextCorpusReader
import numpy as np

print("라이브러리 및 유틸리티 함수를 성공적으로 import했습니다.")


라이브러리 및 유틸리티 함수를 성공적으로 import했습니다.


## 2. 데이터 로드 및 전처리

햄릿 텍스트를 읽어와서 문장들을 준비하고 라벨을 생성합니다.


In [14]:
# 코퍼스 생성
my_corpus = PlaintextCorpusReader("../../", ".*\.txt")
sents = my_corpus.sents(fileids="./data/hamlet.txt")

# 문장들을 문자열로 변환
utts = [" ".join(sent) for sent in sents]

# 라벨 생성: 'be'가 포함된 문장은 True(1), 아니면 False(0)
ys = [sent.count("be") > 0 for sent in sents]

print(f"총 문장 수: {len(utts)}")
print(f"'be'가 포함된 문장 수: {sum(ys)}")
print(f"'be'가 포함되지 않은 문장 수: {len(ys) - sum(ys)}")
print(f"\n첫 번째 문장: {utts[0]}")
print(f"첫 번째 문장 토큰화: {process_utt(utts[0])}")
print(f"첫 번째 라벨: {ys[0]}")
print(f"\n159 번째 문장: {utts[159]}")
print(f"159 번째 문장 토큰화: {process_utt(utts[159])}")
print(f"159 번째 라벨: {ys[159]}")


총 문장 수: 2660
'be'가 포함된 문장 수: 170
'be'가 포함되지 않은 문장 수: 2490

첫 번째 문장: The Project Gutenberg EBook of Hamlet , by William Shakespeare
첫 번째 문장 토큰화: ['the', 'project', 'gutenberg', 'ebook', 'of', 'hamlet', 'by', 'william', 'shakespear']
첫 번째 라벨: False

159 번째 문장: If thou hast any sound , or use of voice ,[ 18 ] Speak to me : If there be any good thing to be done , That may to thee do ease , and grace to me , Speak to me : If thou art privy to thy country ' s fate , Which , happily , foreknowing may avoid , O , speak !
159 번째 문장 토큰화: ['if', 'thou', 'hast', 'ani', 'sound', 'or', 'use', 'of', 'voic', '18', 'speak', 'to', 'me', 'if', 'there', 'be', 'ani', 'good', 'thing', 'to', 'be', 'done', 'that', 'may', 'to', 'thee', 'do', 'eas', 'and', 'grace', 'to', 'me', 'speak', 'to', 'me', 'if', 'thou', 'art', 'privi', 'to', 'thi', 'countri', 's', 'fate', 'which', 'happili', 'foreknow', 'may', 'avoid', 'o', 'speak']
159 번째 라벨: True


## 3. count_utts 함수 구현

각 단어와 라벨 쌍의 빈도를 계산하는 함수를 구현합니다.


In [12]:
def count_utts(result, utts, ys):
    """
    Input:
        result: a dictionary that is used to map each pair to its frequency
        utts: a list of utts
        ys: a list of the sentiment of each utt (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    """

    for y, utt in zip(ys, utts):
        for word in process_utt(utt):
            # define the key, which is the word and label tuple
            pair = (word, y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # if the key is new, add it to the dict and set the count to 1
            else:
                result[pair] = 1

    return result

print("count_utts 함수가 정의되었습니다.")


count_utts 함수가 정의되었습니다.


## 4. 빈도수 계산 및 확인

실제로 단어-라벨 쌍의 빈도를 계산하고 'be' 관련 결과를 확인해봅시다.


In [15]:
# 빈도수 계산
freqs = count_utts({}, utts, ys)

print(f"총 (단어, 라벨) 쌍의 개수: {len(freqs)}")

# 'be' 단어의 빈도 확인
be_true_freq = lookup(freqs, "be", True)
print(f"\n'be'가 True 라벨과 함께 나타나는 빈도: {be_true_freq}")

# 'be'와 관련된 모든 쌍 출력
print("\n'be'가 포함된 모든 (단어, 라벨) 쌍:")
for k, v in freqs.items():
    if "be" in k:
        print(f"{k}: {v}")


총 (단어, 라벨) 쌍의 개수: 5736

'be'가 True 라벨과 함께 나타나는 빈도: 207

'be'가 포함된 모든 (단어, 라벨) 쌍:
('be', True): 207
('be', False): 33


## 5. train_naive_bayes 함수 구현

Naive Bayes 분류기를 훈련하는 함수를 구현합니다. 이 함수는 logprior와 loglikelihood를 계산합니다.
# ![베이즈 정리](베이즈정리.png)


In [16]:
def train_naive_bayes(freqs, train_x, train_y):
    """
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of utts
        train_y: a list of labels correponding to the utts (0,1)
    Output:
        logprior: the log prior.
        loglikelihood: the log likelihood of you Naive bayes equation.
    """
    loglikelihood = {}
    logprior = 0

    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos(전체 positive 빈도) and N_neg(전체 negative 빈도)
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:
            # Increment the number of positive words (word, label)
            N_pos += lookup(freqs, pair[0], True)

        # else, the label is negative
        else:
            # increment the number of negative words (word,label)
            N_neg += lookup(freqs, pair[0], False)

    # Calculate D, the number of documents(문장)
    D = len(train_y)

    # Calculate the number of positive documents(positive 문장 수)
    D_pos = sum(train_y)

    # Calculate the number of negative documents(negative 문장 수)
    D_neg = D - D_pos

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs, word, 1) # positive 문장에서 해당 단어의 빈도
        freq_neg = lookup(freqs, word, 0) # negative 문장에서 해당 단어의 빈도

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V) # P(word|positive)
        p_w_neg = (freq_neg + 1) / (N_neg + V) # P(word|negative)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos / p_w_neg) # log P(word|positive) - log P(word|negative)

    return logprior, loglikelihood

print("train_naive_bayes 함수가 정의되었습니다.")


train_naive_bayes 함수가 정의되었습니다.


## 6. 모델 훈련

Naive Bayes 모델을 훈련하고 결과를 확인해봅시다.


In [17]:
# 모델 훈련
logprior, loglikelihood = train_naive_bayes(freqs, utts, ys)

print(f"Log Prior: {logprior:.4f}")
print(f"어휘집 크기 (log likelihood 개수): {len(loglikelihood)}")

# 몇 가지 단어의 log likelihood 확인
print("\n일부 단어들의 log likelihood:")
sample_words = ['be', 'to', 'the', 'and', 'of']
for word in sample_words:
    if word in loglikelihood:
        print(f"'{word}': {loglikelihood[word]:.4f}")

# 가장 positive한 단어들 (상위 5개)
sorted_words = sorted(loglikelihood.items(), key=lambda x: x[1], reverse=True)
print("\n가장 positive한 단어들 (상위 5개):")
for word, score in sorted_words[:5]:
    print(f"'{word}': {score:.4f}")

# 가장 negative한 단어들 (하위 5개)
print("\n가장 negative한 단어들 (하위 5개):")
for word, score in sorted_words[-5:]:
    print(f"'{word}': {score:.4f}")


Log Prior: -2.6842
어휘집 크기 (log likelihood 개수): 4574

일부 단어들의 log likelihood:
'be': 3.1296
'to': -0.0386
'the': -0.5115
'and': -0.2532
'of': -0.4823

가장 positive한 단어들 (상위 5개):
'be': 3.1296
'equip': 2.7047
'hate': 2.7047
'quiddit': 2.7047
'_doubt': 2.7047

가장 negative한 단어들 (하위 5개):
'_enter_': -2.4428
'horatio': -2.7247
'_and_': -2.7247
'_hor': -3.0383
'h': -3.4774


## 7. naive_bayes_predict 함수 구현

주어진 문장에 대해 예측을 수행하는 함수를 구현합니다.


In [18]:
def naive_bayes_predict(utt, logprior, loglikelihood):
    """
    Input:
        utt: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods + logprior
    """
    # process the utt to get a list of words
    word_l = process_utt(utt)

    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in word_l:
        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    return p

print("naive_bayes_predict 함수가 정의되었습니다.")


naive_bayes_predict 함수가 정의되었습니다.


## 8. 예측 테스트

유명한 햄릿의 대사로 예측을 테스트해봅시다.


In [19]:
# 유명한 햄릿의 대사로 테스트
my_utt = "To be or not to be, that is the question."
p = naive_bayes_predict(my_utt, logprior, loglikelihood)

print(f"테스트 문장: {my_utt}")
print(f"예측 점수: {p:.4f}")
print(f"예측 결과: {'Positive (be 포함될 것으로 예상)' if p > 0 else 'Negative (be 포함되지 않을 것으로 예상)'}")
print(f"실제 결과: {'be가 포함되어 있음' if 'be' in my_utt.lower() else 'be가 포함되어 있지 않음'}")

# 다른 예시들도 테스트
test_sentences = [
    "The king is dead.",
    "What can be done?",
    "Hello world, this is a test.",
    "I will be there soon."
]

print("\n다른 예시들:")
for sentence in test_sentences:
    score = naive_bayes_predict(sentence, logprior, loglikelihood)
    prediction = "Positive" if score > 0 else "Negative"
    actual = "be 포함" if "be" in sentence.lower() else "be 미포함"
    print(f"문장: '{sentence}'")
    print(f"  점수: {score:.4f}, 예측: {prediction}, 실제: {actual}")


테스트 문장: To be or not to be, that is the question.
예측 점수: 2.9321
예측 결과: Positive (be 포함될 것으로 예상)
실제 결과: be가 포함되어 있음

다른 예시들:
문장: 'The king is dead.'
  점수: -4.5880, 예측: Negative, 실제: be 미포함
문장: 'What can be done?'
  점수: -0.4041, 예측: Negative, 실제: be 포함
문장: 'Hello world, this is a test.'
  점수: -3.7019, 예측: Negative, 실제: be 미포함
문장: 'I will be there soon.'
  점수: -0.0185, 예측: Negative, 실제: be 포함


## 9. test_naive_bayes 함수 구현

모델의 정확도를 계산하는 함수를 구현합니다.


In [20]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of utts
        test_y: the corresponding labels for the list of utts
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of utts classified correctly)/(total # of utts)
    """
    accuracy = 0  # return this properly

    y_hats = []
    for utt in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(utt, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error = avg of the abs vals of the diffs between y_hats and test_y
    error = sum(
        [abs(y_hat - test) for y_hat, test in zip(y_hats, test_y)]
    ) / len(y_hats)

    # Accuracy is 1 minus the error
    accuracy = 1 - error

    return accuracy

print("test_naive_bayes 함수가 정의되었습니다.")


test_naive_bayes 함수가 정의되었습니다.


## 10. 모델 성능 평가

훈련 데이터에서 모델의 정확도를 계산해봅시다.


In [21]:
# 모델 정확도 계산
accuracy = test_naive_bayes(utts, ys, logprior, loglikelihood)

print(f"Naive Bayes 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")

# 추가적인 분석을 위한 예측 결과 확인
correct_predictions = 0
total_predictions = 0
true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0

for i, utt in enumerate(utts):
    prediction_score = naive_bayes_predict(utt, logprior, loglikelihood)
    predicted_label = 1 if prediction_score > 0 else 0
    actual_label = ys[i]
    
    total_predictions += 1
    
    if predicted_label == actual_label:
        correct_predictions += 1
        if actual_label == 1:
            true_positives += 1
        else:
            true_negatives += 1
    else:
        if predicted_label == 1 and actual_label == 0:
            false_positives += 1
        elif predicted_label == 0 and actual_label == 1:
            false_negatives += 1

print(f"\n상세 성능 지표:")
print(f"총 예측 수: {total_predictions}")
print(f"정확한 예측 수: {correct_predictions}")
print(f"True Positives: {true_positives}")
print(f"True Negatives: {true_negatives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")

# Precision과 Recall 계산
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nPrecision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


Naive Bayes 정확도: 0.9801 (98.01%)

상세 성능 지표:
총 예측 수: 2660
정확한 예측 수: 2607
True Positives: 124
True Negatives: 2483
False Positives: 7
False Negatives: 46

Precision: 0.9466
Recall: 0.7294
F1 Score: 0.8239


## 11. 일부 예측 결과 상세 분석

몇 가지 예측 결과를 자세히 살펴보겠습니다.


In [22]:
# 몇 가지 예측 결과를 상세히 분석
print("첫 10개 문장의 예측 결과:")
print("-" * 80)
for i in range(10):
    utt = utts[i]
    actual = ys[i]
    score = naive_bayes_predict(utt, logprior, loglikelihood)
    predicted = 1 if score > 0 else 0
    
    print(f"문장 {i+1}: {utt[:60]}{'...' if len(utt) > 60 else ''}")
    print(f"  실제: {actual}, 예측: {predicted}, 점수: {score:.4f}, {'✓' if predicted == actual else '✗'}")
    print()


첫 10개 문장의 예측 결과:
--------------------------------------------------------------------------------
문장 1: The Project Gutenberg EBook of Hamlet , by William Shakespea...
  실제: False, 예측: 0, 점수: -6.5136, ✓

문장 2: This eBook is for the use of anyone anywhere at no cost and ...
  실제: False, 예측: 0, 점수: -6.7460, ✓

문장 3: You may copy it , give it away or re - use it under the term...
  실제: False, 예측: 0, 점수: -16.0860, ✓

문장 4: Title : Hamlet
  실제: False, 예측: 0, 점수: -3.5905, ✓

문장 5: Author : William Shakespeare
  실제: False, 예측: 0, 점수: -3.8117, ✓

문장 6: Editor : Charles Kean
  실제: False, 예측: 0, 점수: -3.4476, ✓

문장 7: Release Date : January 10 , 2009 [ EBook # 27761 ]
  실제: False, 예측: 1, 점수: 1.5141, ✗

문장 8: Language : English
  실제: False, 예측: 0, 점수: -2.3500, ✓

문장 9: Character set encoding : UTF - 8
  실제: False, 예측: 0, 점수: -2.0171, ✓

문장 10: *** START OF THIS PROJECT GUTENBERG EBOOK HAMLET ***
  실제: False, 예측: 0, 점수: -6.5921, ✓



## 12. 모델 파라미터 요약

최종적으로 학습된 모델의 주요 파라미터들을 요약해보겠습니다.


In [23]:
print("=" * 50)
print("Naive Bayes 모델 요약")
print("=" * 50)
print(f"Log Prior: {logprior:.4f}")
print(f"어휘집 크기: {len(loglikelihood)}")
print(f"총 문장 수: {len(utts)}")
print(f"Positive 문장 수: {sum(ys)}")
print(f"Negative 문장 수: {len(ys) - sum(ys)}")
print(f"모델 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print("=" * 50)

# 모델이 가장 중요하게 생각하는 특징들
print("\n모델이 'be 포함'으로 분류하는 데 가장 중요한 단어들:")
for word, score in sorted_words[:10]:
    print(f"  {word}: {score:.4f}")

print("\n모델이 'be 미포함'으로 분류하는 데 가장 중요한 단어들:")
for word, score in sorted_words[-10:]:
    print(f"  {word}: {score:.4f}")


Naive Bayes 모델 요약
Log Prior: -2.6842
어휘집 크기: 4574
총 문장 수: 2660
Positive 문장 수: 170
Negative 문장 수: 2490
모델 정확도: 0.9801 (98.01%)
Precision: 0.9466
Recall: 0.7294
F1 Score: 0.8239

모델이 'be 포함'으로 분류하는 데 가장 중요한 단어들:
  be: 3.1296
  equip: 2.7047
  hate: 2.7047
  quiddit: 2.7047
  _doubt: 2.7047
  juic: 2.7047
  quillet: 2.7047
  plautu: 2.7047
  liest: 2.7047
  seneca: 2.4170

모델이 'be 미포함'으로 분류하는 데 가장 중요한 단어들:
  guildenstern: -1.9774
  _mar: -2.0138
  _exit_: -2.2651
  lord: -2.3642
  l: -2.3870
  _enter_: -2.4428
  horatio: -2.7247
  _and_: -2.7247
  _hor: -3.0383
  h: -3.4774


## 결론

이 노트북에서 우리는:

1. **Naive Bayes 분류기를 처음부터 구현**했습니다
2. **햄릿 텍스트를 사용한 이진 분류 문제**를 해결했습니다
3. **Log Prior와 Log Likelihood 계산** 과정을 이해했습니다
4. **모델의 성능을 다양한 지표로 평가**했습니다
5. **단어별 중요도를 분석**하여 모델의 동작을 이해했습니다

Naive Bayes는 간단하면서도 효과적인 분류 알고리즘으로, 텍스트 분류, 스팸 필터링, 감정 분석 등 다양한 NLP 작업에 널리 사용됩니다. "Naive"라고 불리는 이유는 각 특징(단어)들이 서로 독립적이라는 강한 가정을 하기 때문이지만, 실제로는 이 가정이 성립하지 않아도 놀랍도록 좋은 성능을 보여주는 경우가 많습니다.
